In [3]:
# Cell 1: Setup - Imports and Paths
import scanpy as sc
import pandas as pd
import numpy as np
import os
import warnings
import sys

# Ignore specific warnings often encountered with AnnData
warnings.filterwarnings("ignore", category=FutureWarning, module="anndata")
# Allow viewing more columns
pd.set_option('display.max_columns', 50)

# --- Define paths for the LATEST run ---
RUN_ID = "run_20250501_045924" # From the log file name
BASE_DIR = "/mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq"
STAGE_1_2_DIR = os.path.join(BASE_DIR, "standardized_data", RUN_ID)
STAGE_2_5_DIR = os.path.join(BASE_DIR, "preprocessed_data", RUN_ID)
COMBINED_FILE = os.path.join(STAGE_1_2_DIR, "combined_all_genes_sparse_standardized.h5ad")

print(f"Run ID: {RUN_ID}")
print(f"Stage 1&2 Output Directory: {STAGE_1_2_DIR}")
print(f"Stage 2.5 Output Directory: {STAGE_2_5_DIR}")
print(f"Combined File Path: {COMBINED_FILE}")

# Function to safely load h5ad
def load_h5ad_safe(filepath):
    print(f"\n--- Loading: {filepath} ---")
    if not os.path.exists(filepath):
        print(f"ERROR: File not found!")
        return None
    try:
        adata = sc.read_h5ad(filepath)
        print(f"Successfully loaded. Shape: {adata.shape}")
        return adata
    except Exception as e:
        print(f"ERROR loading file: {e}")
        return None

# Function to inspect adata
def inspect_adata(adata, name="Dataset"):
    if adata is None:
        return
    print(f"\n--- Inspecting {name} ---")
    print("Shape:", adata.shape)

    print("\nOBS (head):")
    try:
        display(adata.obs.head())
    except NameError: # If not in notebook
        print(adata.obs.head().to_string())
    print("\nOBS Columns:", list(adata.obs.columns))
    print("\nOBS Dtypes:\n", adata.obs.dtypes.value_counts())

    print("\nVAR (head):")
    try:
        display(adata.var.head())
    except NameError:
        print(adata.var.head().to_string())
    print("\nVAR Columns:", list(adata.var.columns))
    print("\nVAR Dtypes:\n", adata.var.dtypes.value_counts())


    print("\nUNS Keys:", list(adata.uns.keys()))
    # Look for specific problematic keys from logs
    problem_keys = ['metadata_sources', 'dataset_info']
    for key in problem_keys:
        if key in adata.uns:
             print(f"  UNS['{key}'] type: {type(adata.uns[key])}")
             # Print snippet if it's a dict or list
             if isinstance(adata.uns[key], (dict, list)):
                  print(f"  UNS['{key}'] (snippet): {str(adata.uns[key])[:200]}...")

    # Check specific obs columns known to cause issues
    problem_obs_cols = ['cell_type_info']
    for col in problem_obs_cols:
         if col in adata.obs.columns:
              print(f"  OBS Column '{col}' dtype: {adata.obs[col].dtype}")
              print(f"  OBS Column '{col}' (sample values): {adata.obs[col].unique()[:5]}")

Run ID: run_20250501_045924
Stage 1&2 Output Directory: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/standardized_data/run_20250501_045924
Stage 2.5 Output Directory: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/preprocessed_data/run_20250501_045924
Combined File Path: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/standardized_data/run_20250501_045924/combined_all_genes_sparse_standardized.h5ad


In [4]:
# Cell 2: Inspect Stage 1 Output (V1 files - before Stage 2 processing)
# Note: We expect ENCODE, GTEx, MAGE v1 files might be corrupted due to save errors.
# ADNI v1 should be okay.

adata_adni_v1 = load_h5ad_safe(os.path.join(STAGE_1_2_DIR, "adni_standardized_v1.h5ad"))
inspect_adata(adata_adni_v1, "ADNI_v1 (Stage 1 Output)")

adata_encode_v1 = load_h5ad_safe(os.path.join(STAGE_1_2_DIR, "encode_standardized_v1.h5ad"))
# inspect_adata(adata_encode_v1, "ENCODE_v1 (Stage 1 Output)") # This might fail or show incomplete data

adata_gtex_v1 = load_h5ad_safe(os.path.join(STAGE_1_2_DIR, "gtex_standardized_v1.h5ad"))
# inspect_adata(adata_gtex_v1, "GTEx_v1 (Stage 1 Output)") # This might fail or show incomplete data

adata_mage_v1 = load_h5ad_safe(os.path.join(STAGE_1_2_DIR, "mage_standardized_v1.h5ad"))
# inspect_adata(adata_mage_v1, "MAGE_v1 (Stage 1 Output)") # This might fail or show incomplete data

# You might need to uncomment the inspect_adata calls one by one if they fail due to corruption.
# Focus on whether the file loads at all, and if so, what .obs/.uns looks like.


--- Loading: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/standardized_data/run_20250501_045924/adni_standardized_v1.h5ad ---
Successfully loaded. Shape: (650, 17991)

--- Inspecting ADNI_v1 (Stage 1 Output) ---
Shape: (650, 17991)

OBS (head):


,sample_id,subject_id,dataset,data_type,expression_unit,tissue,platform,processing,sex,age,species,species_ontology,tissue_original,tissue_ontology,data_type_original,assay_ontology,age_original,developmental_stage_ontology
_sample_id,,,,,,,,,,,,,,,,,,
002_S_0413_002_S_0413_gencode_v24_pruned,002_S_0413_002_S_0413_gencode_v24_pruned,002_S_0413,ADNI,Microarray,Normalized intensity,blood,Affymetrix Human Genome U219 Array,gencode_v24,unknown,,human,NCBITaxon:9606,blood,UBERON:0000178,Microarray,,,
002_S_0729_002_S_0729_gencode_v24_pruned,002_S_0729_002_S_0729_gencode_v24_pruned,002_S_0729,ADNI,Microarray,Normalized intensity,blood,Affymetrix Human Genome U219 Array,gencode_v24,unknown,,human,NCBITaxon:9606,blood,UBERON:0000178,Microarray,,,
002_S_1155_002_S_1155_gencode_v24_pruned,002_S_1155_002_S_1155_gencode_v24_pruned,002_S_1155,ADNI,Microarray,Normalized intensity,blood,Affymetrix Human Genome U219 Array,gencode_v24,unknown,,human,NCBITaxon:9606,blood,UBERON:0000178,Microarray,,,
002_S_1261_002_S_1261_gencode_v24_pruned,002_S_1261_002_S_1261_gencode_v24_pruned,002_S_1261,ADNI,Microarray,Normalized intensity,blood,Affymetrix Human Genome U219 Array,gencode_v24,unknown,,human,NCBITaxon:9606,blood,UBERON:0000178,Microarray,,,
002_S_1268_002_S_1268_gencode_v24_pruned,002_S_1268_002_S_1268_gencode_v24_pruned,002_S_1268,ADNI,Microarray,Normalized intensity,blood,Affymetrix Human Genome U219 Array,gencode_v24,unknown,,human,NCBITaxon:9606,blood,UBERON:0000178,Microarray,,,



OBS Columns: ['sample_id', 'subject_id', 'dataset', 'data_type', 'expression_unit', 'tissue', 'platform', 'processing', 'sex', 'age', 'species', 'species_ontology', 'tissue_original', 'tissue_ontology', 'data_type_original', 'assay_ontology', 'age_original', 'developmental_stage_ontology']

OBS Dtypes:
 category    4
object      2
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
Name: count, dtype: int64

VAR (head):


,gene_id,original_ids,gene_name,gene_type,chromosome,mapping_source
ENSG00000000003,ENSG00000000003,ENSG00000000003.14,TSPAN6,protein_coding,chrX,exact_match
ENSG00000000005,ENSG00000000005,ENSG00000000005.5,TNMD,protein_coding,chrX,exact_match
ENSG00000000419,ENSG00000000419,ENSG00000000419.12,DPM1,protein_coding,chr20,exact_match
ENSG00000000457,ENSG00000000457,ENSG00000000457.13,SCYL3,protein_coding,chr1,exact_match
ENSG00000000938,ENSG00000000938,ENSG00000000938.12,FGR,protein_coding,chr1,exact_match



VAR Columns: ['gene_id', 'original_ids', 'gene_name', 'gene_type', 'chromosome', 'mapping_source']

VAR Dtypes:
 object      2
category    1
category    1
category    1
category    1
Name: count, dtype: int64

UNS Keys: ['dataset_info', 'gencode_version', 'harmonized_gencode_version', 'harmonized_reference_genome', 'metadata_validation', 'ontology_mappings', 'original_gencode_version', 'original_reference_genome', 'processing_date', 'reference_genome', 'subject_demographics']
  UNS['dataset_info'] type: <class 'dict'>
  UNS['dataset_info'] (snippet): {'assay_ontology': 'EFO:0002695', 'data_type': 'microarray', 'description': "Alzheimer's Disease Neuroimaging Initiative microarray data with demographic information", 'expression_unit': 'quantile nor...

--- Loading: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/standardized_data/run_20250501_045924/encode_standardized_v1.h5ad ---
Successfully loaded. Shape: (7, 65586)

--- Loading: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq

In [5]:
# Cell 3: Inspect Stage 2 Output (V2 files)
# Note: We expect ENCODE, GTEx, MAGE v2 files might *not exist* or be corrupted
# because the save failed in standardize_metadata.py. ADNI v2 should exist.

adata_adni_v2 = load_h5ad_safe(os.path.join(STAGE_1_2_DIR, "adni_standardized_v2.h5ad"))
inspect_adata(adata_adni_v2, "ADNI_v2 (Stage 2 Output)")

# Check existence of others (likely False)
print("\nChecking for V2 files potentially missing due to save errors:")
print(f"Encode V2 exists: {os.path.exists(os.path.join(STAGE_1_2_DIR, 'encode_standardized_v2.h5ad'))}")
print(f"GTEx V2 exists:   {os.path.exists(os.path.join(STAGE_1_2_DIR, 'gtex_standardized_v2.h5ad'))}")
print(f"MAGE V2 exists:   {os.path.exists(os.path.join(STAGE_1_2_DIR, 'mage_standardized_v2.h5ad'))}")


--- Loading: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/standardized_data/run_20250501_045924/adni_standardized_v2.h5ad ---
Successfully loaded. Shape: (650, 17991)

--- Inspecting ADNI_v2 (Stage 2 Output) ---
Shape: (650, 17991)

OBS (head):


,sample_id,subject_id,dataset,data_type,expression_unit,tissue,platform,processing,sex,age,species,species_ontology,tissue_original,tissue_ontology,data_type_original,assay_ontology,age_original,developmental_stage_ontology,tissue_ontology_confidence
_sample_id,,,,,,,,,,,,,,,,,,,
002_S_0413_002_S_0413_gencode_v24_pruned,002_S_0413_002_S_0413_gencode_v24_pruned,002_S_0413,ADNI,Microarray,Normalized intensity,blood,Affymetrix Human Genome U219 Array,gencode_v24,unknown,,human,NCBITaxon:9606,blood,UBERON:0000178,Microarray,,,,medium
002_S_0729_002_S_0729_gencode_v24_pruned,002_S_0729_002_S_0729_gencode_v24_pruned,002_S_0729,ADNI,Microarray,Normalized intensity,blood,Affymetrix Human Genome U219 Array,gencode_v24,unknown,,human,NCBITaxon:9606,blood,UBERON:0000178,Microarray,,,,medium
002_S_1155_002_S_1155_gencode_v24_pruned,002_S_1155_002_S_1155_gencode_v24_pruned,002_S_1155,ADNI,Microarray,Normalized intensity,blood,Affymetrix Human Genome U219 Array,gencode_v24,unknown,,human,NCBITaxon:9606,blood,UBERON:0000178,Microarray,,,,medium
002_S_1261_002_S_1261_gencode_v24_pruned,002_S_1261_002_S_1261_gencode_v24_pruned,002_S_1261,ADNI,Microarray,Normalized intensity,blood,Affymetrix Human Genome U219 Array,gencode_v24,unknown,,human,NCBITaxon:9606,blood,UBERON:0000178,Microarray,,,,medium
002_S_1268_002_S_1268_gencode_v24_pruned,002_S_1268_002_S_1268_gencode_v24_pruned,002_S_1268,ADNI,Microarray,Normalized intensity,blood,Affymetrix Human Genome U219 Array,gencode_v24,unknown,,human,NCBITaxon:9606,blood,UBERON:0000178,Microarray,,,,medium



OBS Columns: ['sample_id', 'subject_id', 'dataset', 'data_type', 'expression_unit', 'tissue', 'platform', 'processing', 'sex', 'age', 'species', 'species_ontology', 'tissue_original', 'tissue_ontology', 'data_type_original', 'assay_ontology', 'age_original', 'developmental_stage_ontology', 'tissue_ontology_confidence']

OBS Dtypes:
 category    4
object      2
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
Name: count, dtype: int64

VAR (head):


,gene_id,original_ids,gene_name,gene_type,chromosome,mapping_source
ENSG00000000003,ENSG00000000003,ENSG00000000003.14,TSPAN6,protein_coding,chrX,exact_match
ENSG00000000005,ENSG00000000005,ENSG00000000005.5,TNMD,protein_coding,chrX,exact_match
ENSG00000000419,ENSG00000000419,ENSG00000000419.12,DPM1,protein_coding,chr20,exact_match
ENSG00000000457,ENSG00000000457,ENSG00000000457.13,SCYL3,protein_coding,chr1,exact_match
ENSG00000000938,ENSG00000000938,ENSG00000000938.12,FGR,protein_coding,chr1,exact_match



VAR Columns: ['gene_id', 'original_ids', 'gene_name', 'gene_type', 'chromosome', 'mapping_source']

VAR Dtypes:
 object      2
category    1
category    1
category    1
category    1
Name: count, dtype: int64

UNS Keys: ['dataset_info', 'gencode_version', 'harmonized_gencode_version', 'harmonized_reference_genome', 'metadata_validation', 'ontology_mappings', 'original_gencode_version', 'original_reference_genome', 'processing_date', 'reference_genome', 'subject_demographics']
  UNS['dataset_info'] type: <class 'dict'>
  UNS['dataset_info'] (snippet): {'assay_ontology': 'EFO:0002695', 'data_type': 'microarray', 'description': "Alzheimer's Disease Neuroimaging Initiative microarray data with demographic information", 'expression_unit': 'quantile nor...

Checking for V2 files potentially missing due to save errors:
Encode V2 exists: True
GTEx V2 exists:   True
MAGE V2 exists:   True


In [6]:
# Cell 4: Inspect Stage 2.5 Output (Preprocessed files)
# These should exist and reflect the state before combining.

adata_adni_prep = load_h5ad_safe(os.path.join(STAGE_2_5_DIR, "adni_standardized_preprocessed.h5ad"))
inspect_adata(adata_adni_prep, "ADNI Preprocessed (Stage 2.5 Output)")

adata_encode_prep = load_h5ad_safe(os.path.join(STAGE_2_5_DIR, "encode_standardized_preprocessed.h5ad"))
inspect_adata(adata_encode_prep, "ENCODE Preprocessed (Stage 2.5 Output)")
# Check if placeholders were fixed in the *next* step (should still be present here)
if adata_encode_prep and 'gene_id' in adata_encode_prep.var.columns:
    placeholders = adata_encode_prep.var[adata_encode_prep.var['gene_id'].astype(str).str.startswith('PLACEHOLDER_')]
    print(f"\nPlaceholders found in ENCODE preprocessed var['gene_id']: {len(placeholders)}")

adata_gtex_prep = load_h5ad_safe(os.path.join(STAGE_2_5_DIR, "gtex_standardized_preprocessed.h5ad"))
inspect_adata(adata_gtex_prep, "GTEx Preprocessed (Stage 2.5 Output)")

adata_mage_prep = load_h5ad_safe(os.path.join(STAGE_2_5_DIR, "mage_standardized_preprocessed.h5ad"))
inspect_adata(adata_mage_prep, "MAGE Preprocessed (Stage 2.5 Output)")


--- Loading: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/preprocessed_data/run_20250501_045924/adni_standardized_preprocessed.h5ad ---
Successfully loaded. Shape: (650, 17991)

--- Inspecting ADNI Preprocessed (Stage 2.5 Output) ---
Shape: (650, 17991)

OBS (head):


,sample_id,subject_id,dataset,data_type,expression_unit,tissue,platform,processing,sex,age,species,species_ontology,tissue_original,tissue_ontology,data_type_original,assay_ontology,age_original,developmental_stage_ontology,tissue_ontology_confidence
_sample_id,,,,,,,,,,,,,,,,,,,
002_S_0413_002_S_0413_gencode_v24_pruned,002_S_0413_002_S_0413_gencode_v24_pruned,002_S_0413,ADNI,Microarray,Normalized intensity,blood,Affymetrix Human Genome U219 Array,gencode_v24,unknown,,human,NCBITaxon:9606,blood,UBERON:0000178,Microarray,,,,medium
002_S_0729_002_S_0729_gencode_v24_pruned,002_S_0729_002_S_0729_gencode_v24_pruned,002_S_0729,ADNI,Microarray,Normalized intensity,blood,Affymetrix Human Genome U219 Array,gencode_v24,unknown,,human,NCBITaxon:9606,blood,UBERON:0000178,Microarray,,,,medium
002_S_1155_002_S_1155_gencode_v24_pruned,002_S_1155_002_S_1155_gencode_v24_pruned,002_S_1155,ADNI,Microarray,Normalized intensity,blood,Affymetrix Human Genome U219 Array,gencode_v24,unknown,,human,NCBITaxon:9606,blood,UBERON:0000178,Microarray,,,,medium
002_S_1261_002_S_1261_gencode_v24_pruned,002_S_1261_002_S_1261_gencode_v24_pruned,002_S_1261,ADNI,Microarray,Normalized intensity,blood,Affymetrix Human Genome U219 Array,gencode_v24,unknown,,human,NCBITaxon:9606,blood,UBERON:0000178,Microarray,,,,medium
002_S_1268_002_S_1268_gencode_v24_pruned,002_S_1268_002_S_1268_gencode_v24_pruned,002_S_1268,ADNI,Microarray,Normalized intensity,blood,Affymetrix Human Genome U219 Array,gencode_v24,unknown,,human,NCBITaxon:9606,blood,UBERON:0000178,Microarray,,,,medium



OBS Columns: ['sample_id', 'subject_id', 'dataset', 'data_type', 'expression_unit', 'tissue', 'platform', 'processing', 'sex', 'age', 'species', 'species_ontology', 'tissue_original', 'tissue_ontology', 'data_type_original', 'assay_ontology', 'age_original', 'developmental_stage_ontology', 'tissue_ontology_confidence']

OBS Dtypes:
 category    4
object      2
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
Name: count, dtype: int64

VAR (head):


,gene_id,original_ids,gene_name,gene_type,chromosome,mapping_source,original_gene_id,ensembl_id,mapping_confidence
ENSG00000000003,ENSG00000000003,ENSG00000000003.14,TSPAN6,protein_coding,chrX,reference_mapping,ENSG00000000003,ENSG00000000003,high
ENSG00000000005,ENSG00000000005,ENSG00000000005.5,TNMD,protein_coding,chrX,reference_mapping,ENSG00000000005,ENSG00000000005,high
ENSG00000000419,ENSG00000000419,ENSG00000000419.12,DPM1,protein_coding,chr20,reference_mapping,ENSG00000000419,ENSG00000000419,high
ENSG00000000457,ENSG00000000457,ENSG00000000457.13,SCYL3,protein_coding,chr1,reference_mapping,ENSG00000000457,ENSG00000000457,high
ENSG00000000938,ENSG00000000938,ENSG00000000938.12,FGR,protein_coding,chr1,reference_mapping,ENSG00000000938,ENSG00000000938,high



VAR Columns: ['gene_id', 'original_ids', 'gene_name', 'gene_type', 'chromosome', 'mapping_source', 'original_gene_id', 'ensembl_id', 'mapping_confidence']

VAR Dtypes:
 object      4
category    1
category    1
category    1
category    1
category    1
Name: count, dtype: int64

UNS Keys: ['dataset_info', 'gencode_version', 'gene_mapping_stats', 'harmonized_gencode_version', 'harmonized_reference_genome', 'metadata_validation', 'ontology_mappings', 'original_gencode_version', 'original_reference_genome', 'processing_date', 'reference_genome', 'subject_demographics']
  UNS['dataset_info'] type: <class 'dict'>
  UNS['dataset_info'] (snippet): {'assay_ontology': 'EFO:0002695', 'data_type': 'microarray', 'description': "Alzheimer's Disease Neuroimaging Initiative microarray data with demographic information", 'expression_unit': 'quantile nor...

--- Loading: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/preprocessed_data/run_20250501_045924/encode_standardized_preprocessed.h5ad ---
S

...


VAR Columns: ['gene_id', 'original_ids', 'gene_name', 'gene_type', 'chromosome', 'mapping_source', 'original_gene_id', 'ensembl_id', 'mapping_confidence']

VAR Dtypes:
 object      4
category    1
category    1
category    1
category    1
category    1
Name: count, dtype: int64

UNS Keys: ['dataset_info', 'dataset_version', 'gencode_version', 'gene_mapping_stats', 'harmonized_gencode_version', 'harmonized_reference_genome', 'metadata_sources']
  UNS['metadata_sources'] type: <class 'numpy.ndarray'>
  UNS['dataset_info'] type: <class 'dict'>
  UNS['dataset_info'] (snippet): {'data_type': 'RNA-seq', 'expression_unit': 'TPM', 'gencode_version': np.int64(24), 'genes': np.int64(58988), 'samples': np.int64(19616), 'source': 'GTEx', 'subject_count': np.int64(946), 'tissue_coun...

--- Loading: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/preprocessed_data/run_20250501_045924/mage_standardized_preprocessed.h5ad ---
Successfully loaded. Shape: (731, 19428)

--- Inspecting MAGE Preprocess

,sample_id,donor_id,subject_id,tissue,dataset,data_type,expression_unit,is_gencode,is_ensembl,sex,age,species,species_ontology,tissue_original,tissue_ontology,data_type_original,assay_ontology,age_original,developmental_stage_ontology,tissue_ontology_confidence
_sample_id,,,,,,,,,,,,,,,,,,,,
NA06985_NA06985,NA06985_NA06985,NA06985,NA06985,lymphoblast,MAGE,rna-seq,TPM,False,True,unknown,,human,NCBITaxon:9606,lymphoblast,,RNA-seq,EFO:0009922,,,none
NA07000_NA07000,NA07000_NA07000,NA07000,NA07000,lymphoblast,MAGE,rna-seq,TPM,False,True,unknown,,human,NCBITaxon:9606,lymphoblast,,RNA-seq,EFO:0009922,,,none
NA11919_NA11919,NA11919_NA11919,NA11919,NA11919,lymphoblast,MAGE,rna-seq,TPM,False,True,unknown,,human,NCBITaxon:9606,lymphoblast,,RNA-seq,EFO:0009922,,,none
NA11930_NA11930,NA11930_NA11930,NA11930,NA11930,lymphoblast,MAGE,rna-seq,TPM,False,True,unknown,,human,NCBITaxon:9606,lymphoblast,,RNA-seq,EFO:0009922,,,none
NA11932_NA11932,NA11932_NA11932,NA11932,NA11932,lymphoblast,MAGE,rna-seq,TPM,False,True,unknown,,human,NCBITaxon:9606,lymphoblast,,RNA-seq,EFO:0009922,,,none



OBS Columns: ['sample_id', 'donor_id', 'subject_id', 'tissue', 'dataset', 'data_type', 'expression_unit', 'is_gencode', 'is_ensembl', 'sex', 'age', 'species', 'species_ontology', 'tissue_original', 'tissue_ontology', 'data_type_original', 'assay_ontology', 'age_original', 'developmental_stage_ontology', 'tissue_ontology_confidence']

OBS Dtypes:
 category    4
object      3
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
Name: count, dtype: int64

VAR (head):


,gene_id,original_ids,gene_name,gene_type,chromosome,mapping_source,original_gene_id,ensembl_id,mapping_confidence
ENSG00000000003,ENSG00000000003,ENSG00000000003.14,TSPAN6,protein_coding,chrX,reference_mapping,ENSG00000000003,ENSG00000000003,high
ENSG00000000419,ENSG00000000419,ENSG00000000419.12,DPM1,protein_coding,chr20,reference_mapping,ENSG00000000419,ENSG00000000419,high
ENSG00000000457,ENSG00000000457,ENSG00000000457.13,SCYL3,protein_coding,chr1,reference_mapping,ENSG00000000457,ENSG00000000457,high
ENSG00000000460,ENSG00000000460,ENSG00000000460.16,C1orf112,protein_coding,chr1,reference_mapping,ENSG00000000460,ENSG00000000460,high
ENSG00000000938,ENSG00000000938,ENSG00000000938.12,FGR,protein_coding,chr1,reference_mapping,ENSG00000000938,ENSG00000000938,high



VAR Columns: ['gene_id', 'original_ids', 'gene_name', 'gene_type', 'chromosome', 'mapping_source', 'original_gene_id', 'ensembl_id', 'mapping_confidence']

VAR Dtypes:
 object      4
category    1
category    1
category    1
category    1
category    1
Name: count, dtype: int64

UNS Keys: ['dataset_info', 'gene_mapping_stats']
  UNS['dataset_info'] type: <class 'dict'>
  UNS['dataset_info'] (snippet): {'accession': 'PRJNA851328', 'cell_type': 'lymphoblastoid cell lines', 'continental_groups': np.int64(5), 'data_repository': array(['', ''], dtype=object)}...


In [7]:
# Cell 5: Inspect Final Combined Dataset
adata_combined = load_h5ad_safe(COMBINED_FILE)
inspect_adata(adata_combined, "Combined Dataset (Final Output)")

# Specific checks for combined dataset
if adata_combined:
    print("\n--- Combined Dataset Specific Checks ---")
    if 'dataset' in adata_combined.obs.columns:
        print("\nDataset Value Counts:")
        print(adata_combined.obs['dataset'].value_counts())
    else:
        print("\nERROR: 'dataset' column missing in combined obs!")

    if 'present_in_datasets' in adata_combined.var.columns:
         print("\nGene presence sample (first 5):")
         print(adata_combined.var['present_in_datasets'].head())
    else:
         print("\nERROR: 'present_in_datasets' column missing in combined var!")

    print("\nCombined UNS Keys:", list(adata_combined.uns.keys()))
    # Check structure of specific UNS keys if needed
    # print(adata_combined.uns.get('dataset_info'))
    # print(adata_combined.uns.get('dataset_overlap'))


--- Loading: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/standardized_data/run_20250501_045924/combined_all_genes_sparse_standardized.h5ad ---
Successfully loaded. Shape: (21004, 62324)

--- Inspecting Combined Dataset (Final Output) ---
Shape: (21004, 62324)

OBS (head):


,sample_id,subject_id,dataset,data_type,expression_unit,tissue,sex,age
002_S_0413_002_S_0413_gencode_v24_pruned,002_S_0413_002_S_0413_gencode_v24_pruned,002_S_0413,adni,Microarray,Normalized intensity,blood,unknown,
002_S_0729_002_S_0729_gencode_v24_pruned,002_S_0729_002_S_0729_gencode_v24_pruned,002_S_0729,adni,Microarray,Normalized intensity,blood,unknown,
002_S_1155_002_S_1155_gencode_v24_pruned,002_S_1155_002_S_1155_gencode_v24_pruned,002_S_1155,adni,Microarray,Normalized intensity,blood,unknown,
002_S_1261_002_S_1261_gencode_v24_pruned,002_S_1261_002_S_1261_gencode_v24_pruned,002_S_1261,adni,Microarray,Normalized intensity,blood,unknown,
002_S_1268_002_S_1268_gencode_v24_pruned,002_S_1268_002_S_1268_gencode_v24_pruned,002_S_1268,adni,Microarray,Normalized intensity,blood,unknown,



OBS Columns: ['sample_id', 'subject_id', 'dataset', 'data_type', 'expression_unit', 'tissue', 'sex', 'age']

OBS Dtypes:
 object      1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
Name: count, dtype: int64

VAR (head):


,gene_id,present_in_datasets,chromosome,mapping_source,gene_name,original_ids,gene_type
ENSG00000000003,ENSG00000000003,"adni,gtex,mage",chrX,reference_mapping,TSPAN6,ENSG00000000003,protein_coding
ENSG00000000005,ENSG00000000005,"adni,gtex",chrX,reference_mapping,TNMD,ENSG00000000005,protein_coding
ENSG00000000419,ENSG00000000419,"adni,gtex,mage",chr20,reference_mapping,DPM1,ENSG00000000419,protein_coding
ENSG00000000457,ENSG00000000457,"adni,encode,gtex,mage",chr1,reference_mapping,SCYL3,ENSG00000000457,protein_coding
ENSG00000000460,ENSG00000000460,"encode,gtex,mage",chr1,reference_mapping,C1orf112,ENSG00000000460,protein_coding



VAR Columns: ['gene_id', 'present_in_datasets', 'chromosome', 'mapping_source', 'gene_name', 'original_ids', 'gene_type']

VAR Dtypes:
 object      2
category    1
category    1
category    1
category    1
category    1
Name: count, dtype: int64

UNS Keys: ['dataset_info', 'dataset_overlap', 'gene_counts', 'harmonized_gencode_version', 'harmonized_reference_genome', 'sparsity_stats']
  UNS['dataset_info'] type: <class 'dict'>
  UNS['dataset_info'] (snippet): {'approach': 'all_genes_sparse', 'creation_date': '2025-05-01', 'datasets_combined': array(['adni', 'encode', 'gtex', 'mage'], dtype=object), 'description': 'Combined dataset containing all genes from...

--- Combined Dataset Specific Checks ---

Dataset Value Counts:
dataset
gtex      19616
mage        731
adni        650
encode        7
Name: count, dtype: int64

Gene presence sample (first 5):
ENSG00000000003           adni,gtex,mage
ENSG00000000005                adni,gtex
ENSG00000000419           adni,gtex,mage
ENSG000000004